In [132]:
import numpy as np, pandas as pd, xarray as xr
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt

In [133]:



all_dfs = []
data = {}
f = "/home/julienb/Documents/Data/Luisa/Fluorescence.csv"
df = pd.read_csv(f, header=1, index_col=False)
df


,TimeStamp,Markings,CH1-410,CH1-470,CH2-410,CH2-470,Unnamed: 6
0,0.000,NaN,48.834,41.447,17.751,28.828,NaN
1,20.000,NaN,48.839,41.357,17.850,28.849,NaN
2,40.000,NaN,48.718,41.280,17.871,28.811,NaN
3,60.000,NaN,48.615,41.209,17.859,28.759,NaN
4,80.000,NaN,48.540,41.161,17.867,28.752,NaN
...,...,...,...,...,...,...,...
127964,2559292.003,NaN,25.823,26.623,12.299,22.898,NaN
127965,2559312.003,NaN,25.853,26.619,12.333,22.945,NaN
127966,2559332.004,NaN,25.851,26.618,12.331,22.922,NaN
127967,2559352.004,NaN,25.855,26.624,12.307,22.948,NaN


In [134]:
data[str(f)] = {"t": df["TimeStamp"].to_numpy(),"CH-410": df["CH1-410"].to_numpy(), "CH-470": df["CH1-470"].to_numpy()}

def get_input(marking):
    if isinstance(marking, str) and marking:
        if marking.split("*")[2]=='0;':
            return marking.split("*")[0]
    return np.nan
df["file"] = str(f)
df["input"] = df["Markings"].apply(get_input)
df = df.drop(columns=["CH1-470", "CH1-410", "Markings"])
df = df.loc[~df["input"].isna(), :]
all_dfs.append(df)
    
events = pd.concat(all_dfs)
events

,TimeStamp,CH2-410,CH2-470,Unnamed: 6,file,input
223,4460.021,17.113,27.559,NaN,/home/julienb/Documents/Data/Luisa/Fluorescenc...,Input1
631,12620.059,16.784,27.140,NaN,/home/julienb/Documents/Data/Luisa/Fluorescenc...,Input3
657,13140.061,16.748,27.027,NaN,/home/julienb/Documents/Data/Luisa/Fluorescenc...,Input4
669,13380.062,16.769,26.984,NaN,/home/julienb/Documents/Data/Luisa/Fluorescenc...,Input2
889,17780.083,16.641,26.897,NaN,/home/julienb/Documents/Data/Luisa/Fluorescenc...,Input1
...,...,...,...,...,...,...
67551,1351026.303,13.034,23.672,NaN,/home/julienb/Documents/Data/Luisa/Fluorescenc...,Input1
96516,1930329.031,12.694,23.579,NaN,/home/julienb/Documents/Data/Luisa/Fluorescenc...,Input3
96541,1930829.033,12.680,23.475,NaN,/home/julienb/Documents/Data/Luisa/Fluorescenc...,Input4
96588,1931769.038,12.657,23.308,NaN,/home/julienb/Documents/Data/Luisa/Fluorescenc...,Input2


In [135]:
events.groupby("input")["TimeStamp"].count()

input
Input1    60
Input2    52
Input3    59
Input4    52
Name: TimeStamp, dtype: int64

In [136]:
event="Input4"
fiber_ev = events[events["input"] ==event]["TimeStamp"].to_numpy()/1000
fiber_lever_ev = events[events["input"] =="Input3"]["TimeStamp"].to_numpy()/1000
fiber_laser_ev = events[events["input"] =="Input1"]["TimeStamp"].to_numpy()/1000
fiber_ev.shape


(52,)

In [137]:
spike2_channel = "Reward"
spike2_ev: np.ndarray = np.load(f"/home/julienb/Documents/Data/Luisa//Rat26_20240416_newtask21_data/{spike2_channel}.npy")
spike_2_lasertri = np.load("tmp.npy")
if spike2_ev.ndim >1:
    spike2_ev = np.mean(spike2_ev, axis=1)
spike2_ev.shape, spike_2_lasertri.shape

((60,), (61,))

In [138]:
pairs = spike2_ev.reshape(-1, 1) - fiber_ev.reshape(1, -1)
pairs

array([[ 2.32673400e+00, -6.57330700e+00, -2.03333710e+01, ...,
        -6.98976528e+02, -7.09316576e+02, -1.91536224e+03],
       [ 1.12359590e+01,  2.33591800e+00, -1.14241460e+01, ...,
        -6.90067303e+02, -7.00407351e+02, -1.90645301e+03],
       [ 2.50072990e+01,  1.61072580e+01,  2.34719400e+00, ...,
        -6.76295963e+02, -6.86636011e+02, -1.89268167e+03],
       ...,
       [ 1.91998012e+03,  1.91108008e+03,  1.89732001e+03, ...,
         1.21867686e+03,  1.20833681e+03,  2.29114700e+00],
       [ 2.40528970e+03,  2.39638966e+03,  2.38262960e+03, ...,
         1.70398644e+03,  1.69364639e+03,  4.87600732e+02],
       [ 2.40597943e+03,  2.39707939e+03,  2.38331933e+03, ...,
         1.70467617e+03,  1.69433612e+03,  4.88290462e+02]])

In [139]:
sorted = np.sort(pairs.flatten())

In [146]:
a = xr.DataArray(sorted, dims="index")
a["index"] = np.arange(sorted.size)
win_size = fiber_ev.size-1
var = a.rolling(index=win_size, center=True).var()
i = var.argmin("index")
d = a.sel(index=i).item()
dict(shift=d, precision=(a.sel(index=i+win_size//2) - a.sel(index=i-win_size//2)).item())


{'shift': 2.3335449999999014, 'precision': 0.0314649999998835}

In [141]:
# basic_h, basic_edges = np.histogram(pairs.flatten(), bins=np.linspace(-100, 100, 200))
# n = basic_h.max()
# pos = basic_h.argmax()
# print(n)
# h, edges = np.histogram(pairs.flatten(), range=(basic_edges[pos], basic_edges[pos+1]), bins=1000)
# pos = h.argmax()
# d = (edges[pos]+ edges[pos+1])/2
# print(d, h.max())

In [142]:

plt.eventplot(fiber_ev, color="red", label=f"fiber {event}")
plt.eventplot(spike2_ev - d, color="green", label=f"spike2 {spike2_channel}", linestyles=":")
plt.eventplot(spike_2_lasertri - d, color="blue", label=f"spike2 lasertri", linestyles=":")
plt.eventplot(fiber_lever_ev, color="purple", label=f"fiber lever_press?")
plt.eventplot(fiber_laser_ev, color="pink", label=f"fiber input1")
plt.legend()